# 1) DATENSAMMLUNG

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
headers = ['brand','model', 'price', 'year', 'transmission', 'fuelConsumption', 'tax', 'mileage', 'fueltype', 'color', 'variant']
df = pd.DataFrame(columns = headers)


In [ ]:
colors = ['beige', 'black', 'blue', 'bronze', 'brown', 'gold', 'green', 'grey', 'orange', 'purple', 'red', 'silver', 'turquoise', 'white', 'yellow']
cars = ['toyota', 'volvo', 'audi', 'bmw', 'citroen', 'ford', 'honda', 'hyundai', 'kia', 'nissan']
url = "https://www.exchangeandmart.co.uk/used-cars-for-sale/"

In [ ]:
for color in colors:
    for pageNumber in range(1,100):
        for car in cars:
            currentUrl = url + car + "/any-distance-from-se15-1la/colour-" + color + '/page' + str(pageNumber)
            page = requests.get(currentUrl)
            soup = BeautifulSoup(page.text, 'html')
            items = soup.find_all('div', attrs={"class" : "result-item"})

            if items[0].find('span', attrs={"class" : "ttl_mk"}) is None:
                break
            
            for item in items:
                brand = item.find('span', attrs={"class" : "ttl_mk"}).text
                model = item.find('span', attrs={"class" : "ttl_md"}).text
                price = item.find('span', attrs={"class" : "price--primary"}).text
                variant = item.find('div', attrs={"class" : "result-item__variant"}).text
                keyDetails = item.find_all('p', attrs={"class" : "key-details__item"})
                if(len(keyDetails) < 6):
                    continue
                year = keyDetails[0].text
                transmission = keyDetails[1].text
                fuelConsumption = keyDetails[2].text
                tax = keyDetails[3].text
                mileage = keyDetails[4].text
                fueltype = keyDetails[5].text
                data = [brand, model, price, year, transmission, fuelConsumption, tax, mileage, fueltype, color, variant]
                df.loc[len(df)-1] = data

In [ ]:
df.to_csv('AllData.csv')
groupedDFs = [v for k, v in df.groupby('brand')]
for brandDF in groupedDFs:
    brandDF.to_csv(''+brandDF.brand.iloc[0]+'.csv') 

# 2) DATENBEREINIGUNG

**Was beschreiben die Daten?**

**Warum sind die Daten relevant oder interessant?**

**Welche Daten sind enthalten?**

**Wie sind die Daten formatiert?**

**Welche Schritte haben Sie unternommen, um die Daten zu bereinigen?**

**Gibt es Ausreißer, Duplikate oder fehlende Werte?**

**Gibt es Fehler im Datensatz?**

**Sind die Daten konsistent?**

**Welche Besonderheiten sind Ihnen aufgefallen?**

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import statsmodels.api as sm

In [37]:
df= pd.read_csv('AllData.csv')
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37147 entries, 0 to 37146
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       37147 non-null  int64  
 1   brand            37147 non-null  object 
 2   model            37147 non-null  object 
 3   price            37147 non-null  object 
 4   year             37056 non-null  float64
 5   transmission     37147 non-null  object 
 6   fuelConsumption  37147 non-null  object 
 7   tax              37147 non-null  object 
 8   mileage          37147 non-null  object 
 9   fueltype         37147 non-null  object 
 10  color            37147 non-null  object 
 11  variant          36549 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 3.4+ MB


(37147, 12)

In [6]:
df.head()

,Unnamed: 0,brand,model,price,year,transmission,fuelConsumption,tax,mileage,fueltype,color,variant
0,-1,Toyota,Aygo,"£20,430",2022.0,Automatic,56.5 mpg,Tax: £180,"Mileage: 4,497",Petrol,beige,o X 1.0 VVT-i Exclusive 5dr Auto
1,0,Toyota,Aygo,"£17,998",2023.0,Automatic,56.5 mpg,Tax: £180,"Mileage: 2,345",Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
2,1,Toyota,Aygo,"£14,225",2022.0,Manual,58.9 mpg,Tax: £180,"Mileage: 36,729",Petrol,beige,o X 1.0 VVT-i Edge 5dr
3,2,Toyota,Aygo,"£18,431",2023.0,Automatic,56.5 mpg,Tax: £180,"Mileage: 3,620",Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
4,3,Toyota,Aygo,"£17,159",2022.0,Manual,57.7 mpg,Tax: £180,"Mileage: 4,000",Petrol,beige,o X 1.0 VVT-i Exclusive 5dr


In [ ]:
#Formatting the price data / Bereinigung der Preisdaten
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].str.replace('£','')
df['price'] = pd.to_numeric(df['price'])
#Formatting the year data 
df["year"].dropna(inplace=True)
df['year'] = df['year'].astype('int', errors='ignore')
#Formatting the fuelConsumption data 
df['fuelConsumption'] = df['fuelConsumption'].str.replace(' mpg','')
df['fuelConsumption'] = pd.to_numeric(df['fuelConsumption'])
#Formatting the Tax data 
df['tax'] = df['tax'].str.replace('Tax: n/a','')
df['tax'] = df['tax'].str.replace('Tax: £','')
df['tax'] = pd.to_numeric(df['tax'])
#Formatting the mileage data 
df['mileage'] = df['mileage'].str.replace(',','')
df['mileage'] = df['mileage'].str.replace('Mileage: ','')
#df['mileage'] = df['mileage'].str.replace('Mileage: Unknown','')
df['mileage'] = df['mileage'].str.replace('Unknown','')
df['mileage'] = pd.to_numeric(df['mileage'])

In [39]:
df.head()
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37147 entries, 0 to 37146
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       37147 non-null  int64  
 1   brand            37147 non-null  object 
 2   model            37147 non-null  object 
 3   price            37147 non-null  int64  
 4   year             37056 non-null  float64
 5   transmission     37147 non-null  object 
 6   fuelConsumption  37147 non-null  float64
 7   tax              36646 non-null  float64
 8   mileage          37136 non-null  float64
 9   fueltype         37147 non-null  object 
 10  color            37147 non-null  object 
 11  variant          36549 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 3.4+ MB


(37147, 12)

In [40]:
pd.isnull(df).sum()

Unnamed: 0           0
brand                0
model                0
price                0
year                91
transmission         0
fuelConsumption      0
tax                501
mileage             11
fueltype             0
color                0
variant            598
dtype: int64

In [41]:
df["tax"].fillna(0, inplace=True)
df["mileage"].fillna(df["mileage"].mean(), inplace=True)
df["year"].fillna(df["year"].mean(), inplace=True)
df["variant"].fillna("unknown variant", inplace=True)

In [42]:
pd.isnull(df).sum()

Unnamed: 0         0
brand              0
model              0
price              0
year               0
transmission       0
fuelConsumption    0
tax                0
mileage            0
fueltype           0
color              0
variant            0
dtype: int64